In [1]:
import pandas
import json
from bokeh.models import CustomJS, Slider
from bokeh.io import show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.layouts import row

In [2]:
def get_medians(zipcode, zipcode_df):
    try:
        median = int(zipcode_df[zipcode]['Number of Returns'].iloc[0]/2)
        for i in range(1, 7):
            if zipcode_df[zipcode]['Number of Returns'].iloc[i] < median:
                median -= zipcode_df[zipcode]['Number of Returns'].iloc[i]
            else:
                return zipcode_df[zipcode]['AGI'].iloc[i]
    except:
        return "No Information"

In [3]:
COLORMAP = {'$1 under $25,000': '#0868ac',
            '$25,000 under $50,000': '#43a2ca',
            '$50,000 under $75,000': '#7bccc4',
            '$75,000 under $100,000': '#a8ddb5',
            '$100,000 under $200,000': '#ccebc5',
            '$200,000 or more': '#f0f9e8',
            'No Information': '#7a807e', }

In [4]:
neighborhood_raw = pandas.read_excel("neighborhoods.xlsx")

In [6]:
zip_codes_neighborhoods = {}
for idx, zipcodes in enumerate(neighborhood_raw["Zip Codes"]):
    if type(zipcodes) == str:
        for zipcode in zipcodes.split(","):
            zip_codes_neighborhoods[zipcode.strip(
            )] = neighborhood_raw["Neighborhood"][idx]
    else:
        zip_codes_neighborhoods[str(
            zipcodes)] = neighborhood_raw["Neighborhood"][idx]

In [47]:
# IRS Data
def get_income_data(year):
    year = str(year)
    df = pandas.read_csv(f"income_data/formatted_data/{year}_irs_data")
    return df

df = get_income_data(2014)

True

In [55]:
# Map Data
zip_dfs = {}

with open('nyc_zip_codes.json') as json_file:
    data = json.load(json_file)
    for i in range(len(data['features'])):
        zipcode = data['features'][i]['properties']['postalCode']
        if int(zipcode) in df['Zip Code'].values:
            zip_dfs[zipcode] = df.loc[df['Zip Code'].values == int(zipcode)]
        try:
            data['features'][i]['properties']['neighborhood'] = zip_codes_neighborhoods[zipcode]
        except:
            data['features'][i]['properties']['neighborhood'] = "None"

        data['features'][i]['properties']['median_income'] = get_medians(
            zipcode, zip_dfs).strip()
        data['features'][i]['properties']['color'] = COLORMAP[data['features']
                                                              [i]['properties']['median_income']]

geo_source = GeoJSONDataSource(geojson=json.dumps(data))

   Zip Code                      AGI  Number of Returns
0     10001                      NaN              14080
1     10001         $1 under $25,000               3880
2     10001    $25,000 under $50,000               2530
3     10001    $50,000 under $75,000               1850
4     10001   $75,000 under $100,000               1300
5     10001  $100,000 under $200,000               2350
6     10001         $200,000 or more               2170
